# Project
This is the main file for project 2 in COMP 472.

**Team:** Deus Ex Machina

**Member(s):** Sobhan Mehrpour Kevishahi - 40122438

**Github Repository :** [https://github.com/Sobhan-M/comp472-project2](https://github.com/Sobhan-M/comp472-project2)

## Implementation Of Algorithms
In this section I will be implementing the different search algorithms as methods. The implemented classes and functions have been included in separate Python files and are used to abstract away tedious aspects of the implementation.

In [1]:
# I realize this way of importing is not good practice, but I know there aren't any conflicts so I'm doing it anyway.

from car import *
from grid import *
from position import *
from node import *
from priorityqueue import *
from reporter import *
from heuristics import *
from iomanager import *

import numpy as np
import time

In [2]:
example1 = "IIB...C.BHHHC.AAD.....D.EEGGGF.....F"
example2 = "C.B...C.BHHHAADD........EEGGGF.....F"
example3 = "...GF...BGF.AABCF....CDD...C....EE.."
example4 = "....F...B.F.AABCF....C.....C....EE.."

examples = [example1, example2, example3, example4]

### Uniform Cost Search
This is the implementation of uniform cost search. This algorithm only uses the cost (distance from root node in this case) when choosing which node to expand.

In [3]:
def uniformCostSearch(puzzleLine:str, reportNumber=0, outputFileName="ucs", shouldWriteReport=True):
	"""
	Performs uniform cost search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: n.cost()) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: n.cost(), lambda n: 0)

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Update node if needed.
				if child.cost() < openList.getValue(child).cost():
					openList.updateValue(child)
			else:
				openList.insert(child)

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

In [4]:
# for i in range(4):
# 	uniformCostSearch(examples[i], i+1, "example-reports/ucs-example")

### Greedy Best First Search
In this implementation we use a heuristic to determine which nodes to explore. This does not guarantee an optimal solution.

In [5]:
def greedyBestFirstSearch(puzzleLine:str, heuristic, reportNumber=0, outputFileName="gbfs", shouldWriteReport=True):
	"""
	Performs greedy best first search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(lambda n: heuristic(n)) # The minimizing function is simply the heuristic.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			if child.isGoal():
				goalNode = child
				break

			if closedList.get(str(child)) is not None: # Avoid visiting already visited nodes.
				continue

			if openList.getValue(child) is not None: # Avoid visiting a child node already in the open list. Heuristics don't change.
				continue

			openList.insert(child)

		if goalNode is not None:
			# Counting the child if it's the goal node.
			reporter.countVisit()
			reporter.addToSearchPath(visiting, lambda n: 0, lambda n: 0, lambda n: heuristic(n))
			break

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

	

In [6]:
# for i in range(4):
# 	greedyBestFirstSearch(examples[i], h1, i+1, "example-reports/gbfs-example")

### Algorithm A Search
This implementation of algorithm A considers both the cost and the heuristic when choosing which node to visit.

In [7]:
def algorithmA(puzzleLine:str, heuristic, reportNumber=0, outputFileName="a", shouldWriteReport=True):
	"""
	Performs greedy best first search to find the solution of the puzzle.
	Creates the solution file and search file, based on the given name.
	Returns the goal node.
	"""

	# Preparing report.
	reporter = Reporter(puzzleLine)
	reporter.startTimer()

	# Setting things up.
	f = lambda n: heuristic(n) + n.cost()
	h = lambda n: heuristic(n)
	g = lambda n: n.cost()

	root = generateStartNode(puzzleLine)
	openList = PriorityQueue(f) # The minimizing function is simply the cost.
	openList.insert(root)
	closedList = dict()
	goalNode = None

	# Main loop.
	while not openList.isEmpty():

		visiting = openList.removeMin()
		reporter.countVisit()
		reporter.addToSearchPath(visiting, f, g, h)

		if visiting.isGoal():
			goalNode = visiting
			break
		else:
			closedList[str(visiting)] = visiting

		children = visiting.expandChildren()

		for child in children:
			
			closedNode = closedList.get(str(child))
			openNode = openList.getValue(child)
			
			if closedNode is not None: # Handling the closed list.
				if f(child) < f(closedNode):
					closedList.pop(str(child))
					openList.insert(child)
				continue
			elif openNode is not None: # Handling open list.
				if f(child) < f(openNode):
					openList.updateValue(child)
				continue
			else:
				openList.insert(child)

	# Finalizing report.
	reporter.endTimer()
	reporter.setGoalNode(goalNode)
	report = reporter.generateSolutionReport()

	if shouldWriteReport:
		solutionFileName = f"{outputFileName}-sol-{reportNumber}.txt"
		searchFileName = f"{outputFileName}-search-{reportNumber}.txt"

		with open(solutionFileName, "w") as file:
			file.write(report)

		with open(searchFileName, "w") as file:
			file.write("\n".join(reporter.searchPath))

	return goalNode, reporter

	

In [8]:
# for i in range(4):
# 	algorithmA(examples[i], h5, i+1, "example-reports/a-example-h5")

## Report Generation
Takes in a file and writes reports for each puzzle. This includes all solution files and search files.

In [9]:
def generateReports(fileName, outputDirectory=""):
	puzzles = extractPuzzleLines(fileName)

	for i in range(len(puzzles)):
		# startTime = time.time()
		
		uniformCostSearch(puzzles[i], i, outputDirectory + "ucs")

		greedyBestFirstSearch(puzzles[i], h1, i, outputDirectory + "gbfs-h1")
		greedyBestFirstSearch(puzzles[i], h2, i, outputDirectory + "gbfs-h2")
		greedyBestFirstSearch(puzzles[i], h3, i, outputDirectory + "gbfs-h3")
		greedyBestFirstSearch(puzzles[i], h4, i, outputDirectory + "gbfs-h4")
		greedyBestFirstSearch(puzzles[i], h5, i, outputDirectory + "gbfs-h5")

		algorithmA(puzzles[i], h1, i, outputDirectory + "a-h1")
		algorithmA(puzzles[i], h2, i, outputDirectory + "a-h2")
		algorithmA(puzzles[i], h3, i, outputDirectory + "a-h3")
		algorithmA(puzzles[i], h4, i, outputDirectory + "a-h4")
		algorithmA(puzzles[i], h5, i, outputDirectory + "a-h5")

		# endTime = time.time()	
		# print(f"Done with item {i} after {endTime - startTime} s.")

In [10]:
# generateReports("input/sample-input.txt", "output/samples/")

## Analysis Generation
We can create a function that generates a CSV file that can easily be exported into a spreadsheet for more analysis.

In [11]:
def generateLine(puzzleNum:int, algorithm:str, heuristic:str, reporter:Reporter):
	searchPathLength = reporter.nodesVisited
	executionTime = reporter.timeElapsed()

	if reporter.goalNode is None:
		solutionLength = "N/A"
	else:
		solutionLength = reporter.countTotalMoves()

	return "{},{},{},{},{},{}".format(puzzleNum, algorithm, heuristic, solutionLength, searchPathLength, executionTime)

In [12]:
def generateSpreadsheet(inputFileName, outputFileName):
	puzzles = extractPuzzleLines(inputFileName)
	results = list()
	results.append("Puzzle Number,Algorithm,Heuristic,Length of the Solution,Length of the Search Path,Execution Time (in seconds)")
	
	for i in range(len(puzzles)):
		# startTime = time.time()

		reporter1 = uniformCostSearch(puzzles[i], i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "UCS", "N/A", reporter1))

		reporter1 = greedyBestFirstSearch(puzzles[i], h1, i, shouldWriteReport=False)[1]
		reporter2 = greedyBestFirstSearch(puzzles[i], h2, i, shouldWriteReport=False)[1]
		reporter3 = greedyBestFirstSearch(puzzles[i], h3, i, shouldWriteReport=False)[1]
		reporter4 = greedyBestFirstSearch(puzzles[i], h4, i, shouldWriteReport=False)[1]
		reporter5 = greedyBestFirstSearch(puzzles[i], h5, i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "GBFS", "h1", reporter1))
		results.append(generateLine(i, "GBFS", "h2", reporter2))
		results.append(generateLine(i, "GBFS", "h3", reporter3))
		results.append(generateLine(i, "GBFS", "h4", reporter4))
		results.append(generateLine(i, "GBFS", "h5", reporter5))

		reporter1 = algorithmA(puzzles[i], h1, i, shouldWriteReport=False)[1]
		reporter2 = algorithmA(puzzles[i], h2, i, shouldWriteReport=False)[1]
		reporter3 = algorithmA(puzzles[i], h3, i, shouldWriteReport=False)[1]
		reporter4 = algorithmA(puzzles[i], h4, i, shouldWriteReport=False)[1]
		reporter5 = algorithmA(puzzles[i], h5, i, shouldWriteReport=False)[1]
		results.append(generateLine(i, "A/A*", "h1", reporter1))
		results.append(generateLine(i, "A/A*", "h2", reporter2))
		results.append(generateLine(i, "A/A*", "h3", reporter3))
		results.append(generateLine(i, "A/A*", "h4", reporter4))
		results.append(generateLine(i, "A/A*", "h5", reporter5))

		# endTime = time.time()	
		# print(f"Done with item {i} after {endTime - startTime} s.")

	with open(outputFileName + ".csv", "w") as file:
		file.write("\n".join(results))

In [13]:
# generateSpreadsheet("input/sample-input.txt", "output/sample-analysis")

### Puzzles
Here I randomly generate some puzzles to use.

In [14]:
from randompuzzle import generateRandomPuzzle

puzzles = list()
for i in range(50):
	puzzles.append(generateRandomPuzzle())

with open("input/generated-puzzles-2.txt", "w") as file:
	file.write("\n".join(puzzles))

Now I will create the analysis of these puzzles.

In [15]:
generateSpreadsheet("input/generated-puzzles-2.txt", "output/analysis-2")

### Analysis Results

All this information can be easily accessed and viewed in `output/analysis.xlsx`.

#### 1. Solution Length

- **Uniform Cost Search:** Has LOWEST cost.
- **Algorithm A/A*:** Often has lowest cost, but not guaranteed. Except when admissible.
- **Greedy Best First Search:** No guarantee of lowest cost.

For puzzles, we had the lowest cost solution with **Uniform Cost Search** which is to be expected. In almost all cases, **Algorithm A/A\*** also shared the lowest cost, except with `h3` which is to be expected as that heuristic is not admissible.

In many puzzles, **Greedy Best First Search** also had the lowest cost (especially if the solution path is short), but it was never guaranteed. Again, this matches our expectation of the algorithm.

#### 2. Heuristic Admissibility

- **h1:** The first heuristic **_is admissible_** because it counts the number of cars in the path of A. In order for A to get to the exit we must make at least one move per blocking car in order to open the path. Then A must make a final move to reach the exit. Therefore this heuristic is admissible and it will result in **Algorithm A/A\*** guaranteeing the optimal solution.

- **h2:** The second heuristic **_is NOT admissible_** because it counts the number of blocked position. To understand why, let us consider this simple example:
```
. . . . . .
. . . . . .
A A B B B .
. . . . . .
. . . . . .
. . . . . .
```
The heuristic would be 3 in this instance, but we reach the solution in only 2 moves. The first move is from B to the right, which removes it from the grid. Then A reaches the exit in the next move. This shows that the heuristic is not admissible.

- **h3:** The third heuristic **_is NOT admissible_** (assuming the constant must be an integer greater than 1). Consider the following example:
```
. . . . . .
. . B C . .
A A B C . .
. . . . . .
. . . . . .
. . . . . .
```
We have 2 blocking vehicles, and the smallest constant would be 2. Therefore this heuristic gives us a value of 4. But we see that we can solve this puzzle in only 3 moves. B moves up for the first move. C moves up for the second move. Finally, A moves to the exit for the third move. This shows the heuristic is not admissible.

- **h4:** This heuristic **_is admissible_**. The heuristic is designed to consider the distance of A to the exit to encourage moving A closer to the exit. This is because the other heuristics return a value of 0 even if we have not reached the goal state. This heuristic solves that issue by counting the distance to the exit cell and dividing it by 4. We divide by 4 to ensure admissibility, because we can move a maximum of 4 cells in a move. Therefore, we get 4/4=1 at the farthest point, then 3/4, 2/4, and 1/4 for intermediate points. Finally 0 when we are at the goal state.

- **h5:** This heuristic **_is admissible_**. It is a combination of h1 and h4 and perfectly describes the situation mentioned when analysing h1. It considers both the blocking cars and the distance to the exit. And because blocking cars have more weight than moving closer to the exit, the heuristic will prioritize that. Once all the cars are out of the way, this heuristic will allow us to instantly find the solution, unlike h1, h2, and h3. 

#### 3. Execution Time

Consider the following graphs of the data:
- **Note:** The issue of negative times around puzzle 23 is a problem with the interpolating polynomial, not the data itself.

<img src="output/solution-times-1.png" height=500px>
<img src="output/solution-times-2.png" height=500px>

In this case, we notice that an uninformed search (Uniform Cost Search) is often (but not always) one of the slowest algorithms. This is especially evident in puzzle 44, where UCS has a huge spike compared to other algorithms.

Within a given algorithm, there seems to be little difference based on which heuristic is used. In some cases, certain heuristics will cause a sudden jump. But there doesn't appear to be any notable differences between them.



#### 4. Search Paths

Looking at the image below, it is interesting to note that the size of the search path differs drastically between algorithms. With **Uniform Cost Search** often having the largest search path and **Greedy Best First Search** often having the smallest. This falls in line with our expectations, as Uniform Cost Search systematically checks all low cost solution paths, even if they do not yield a solution. In addition, both Uniform Cost Search and Algorithm A/A* also reinsert certain nodes if the new cost is smaller, which means the algorithm could search the same node several times. Whereas Greedy Best First Search only ever searches an algorithm once.

<img src="output/search-paths-1.png" height=500px>

It is also interesting to note that heuristic 4 tends to search more nodes than other algorithms. This may be due to the fact that this heuristic acts like Uniform Cost Search until the path to the exit is clear.

<img src="output/search-paths-2.png" height=500px>
<img src="output/search-paths-3.png" height=500px>